In [9]:
import torch
import pathpyG as pp
from collections import defaultdict

from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index
from torch_geometric.data import Data

from pathpyG import Graph, TemporalGraph, MultiOrderModel

pp.config['torch']['device'] = 'cuda'

In [10]:
tedges = [('a', 'b', 1),('a', 'b', 2), ('b', 'a', 3), ('b', 'c', 3), ('d', 'c', 4), ('a', 'b', 4), ('c', 'b', 4),
              ('c', 'd', 5), ('b', 'e', 5), ('c', 'b', 6)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.mapping)
print(t.N)
print(t.M)

a -> 0
b -> 1
c -> 2
d -> 3
e -> 4

5
10


In [11]:
t = pp.TemporalGraph.from_csv('../data/ants_1_1.tedges')
print(t)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	t		<class 'torch.Tensor'> -> torch.Size([1911])
	src		<class 'torch.Tensor'> -> torch.Size([1911])
	dst		<class 'torch.Tensor'> -> torch.Size([1911])



In [12]:
m = pp.MultiOrderModel.from_temporal_graph(t, max_order=4, delta=2)
print(m.layers[2])
for i, e in enumerate(m.layers[2].edges):
    print(e, m.layers[2].data.edge_weight[i])

Directed graph with 947 nodes and 262 edges

Node attributes
	node_sequence		<class 'torch.Tensor'> -> torch.Size([947, 2])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([262])

Graph attributes
	num_nodes		<class 'int'>

(('Y_WY', 'WG_R'), ('WG_R', 'Y_WY')) tensor(1., device='cuda:0')
(('Y_WY', 'WG_R'), ('WG_R', '____topleft')) tensor(1., device='cuda:0')
(('Y_WY', '_W__'), ('_W__', 'Y_WY')) tensor(1., device='cuda:0')
(('Y_WY', '____pale'), ('____pale', 'Y_WY')) tensor(1., device='cuda:0')
(('WRBB', 'WG_R'), ('WG_R', 'WRBB')) tensor(1., device='cuda:0')
(('WRBB', 'YYGGmid'), ('YYGGmid', 'WRBB')) tensor(1., device='cuda:0')
(('WRBB', 'WGBB'), ('WGBB', 'WYGG')) tensor(1., device='cuda:0')
(('WRBB', 'GR__'), ('GR__', 'WRBB')) tensor(1., device='cuda:0')
(('WRBB', 'RWWG'), ('RWWG', 'WRBB')) tensor(1., device='cuda:0')
(('WRR_', 'GGRR'), ('GGRR', 'WRR_')) tensor(3., device='cuda:0')
(('WRR_', 'G_GW'), ('G_GW', 'WRR_')) tensor(1., device='cuda:0')
(('WRR_', 'YGWW'), (

In [13]:
def temporal_shortest_paths(g: pp.TemporalGraph, delta, max_k) -> tuple[torch.Tensor, dict[int, torch.Tensor]]:
    """
    Calculates all shortest paths between all pairs of nodes
    based on a set of empirically observed paths.
    """
    shortest_paths = {}
    shortest_path_lengths = torch.full((g.N, g.N), float("inf"), device=g.data.edge_index.device)
    shortest_path_lengths.fill_diagonal_(float(0))
    out_degree = degree(g.data.edge_index[0], num_nodes=g.N)
    in_degree = degree(g.data.edge_index[1], num_nodes=g.N)

    # Set combinations where a path is impossible to -1 to improve stopping conditions
    shortest_path_lengths[out_degree == 0] = -1
    shortest_path_lengths[:, in_degree == 0] = -1

    # first-order edge index
    edge_index, timestamps = sort_edge_index(g.data.edge_index, g.data.t)
    node_sequence = torch.arange(g.data.num_nodes, device=edge_index.device).unsqueeze(1)
    
    # second-order edge index with time-respective filtering
    k = 2
    null_model_edge_index = pp.MultiOrderModel.lift_order_edge_index(
        edge_index, num_nodes=node_sequence.size(0)
    )
    # Update node sequences
    node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
    # Remove non-time-respecting higher-order edges
    time_diff = timestamps[null_model_edge_index[1]] - timestamps[null_model_edge_index[0]]
    non_negative_mask = time_diff > 0
    delta_mask = time_diff <= delta
    time_respecting_mask = non_negative_mask & delta_mask
    edge_index = null_model_edge_index[:, time_respecting_mask]

    # Use node sequences to update shortest path lengths
    def update_paths(node_sequence, k):
        path_starts = node_sequence[:, 0]
        path_ends = node_sequence[:, -1]
        mask = shortest_path_lengths[path_starts, path_ends] >= k - 1
        shortest_path_lengths[path_starts[mask], path_ends[mask]] = k - 1
        shortest_paths[k-1] = torch.unique(node_sequence[mask], dim=0)
    update_paths(node_sequence, k)

    k = 3
    while torch.max(shortest_path_lengths) > k and edge_index.size(1) > 0 and k < max_k:
        print(f"k = {k}, edge_index size = {edge_index.size(1)}")
        ho_index = MultiOrderModel.lift_order_edge_index(edge_index, num_nodes=node_sequence.size(0))
        node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
        edge_index = ho_index
        print("\tOrder lifted.")
        update_paths(node_sequence, k)
        print("\tPaths updated.")
        k += 1
    
    shortest_path_lengths[shortest_path_lengths == -1] = float("inf")
    return shortest_path_lengths, shortest_paths

In [14]:
sp_lengths, sp = temporal_shortest_paths(t, delta=30, max_k=60)
# print(sp_lengths)
# print(sp)

k = 3, edge_index size = 2518
	Order lifted.
	Paths updated.
k = 4, edge_index size = 3572
	Order lifted.
	Paths updated.
k = 5, edge_index size = 5310
	Order lifted.
	Paths updated.
k = 6, edge_index size = 8033
	Order lifted.
	Paths updated.
k = 7, edge_index size = 12047
	Order lifted.
	Paths updated.
k = 8, edge_index size = 18218
	Order lifted.
	Paths updated.
k = 9, edge_index size = 26365
	Order lifted.
	Paths updated.
k = 10, edge_index size = 36928
	Order lifted.
	Paths updated.
k = 11, edge_index size = 50536
	Order lifted.
	Paths updated.
k = 12, edge_index size = 67043
	Order lifted.
	Paths updated.
k = 13, edge_index size = 88824
	Order lifted.
	Paths updated.
k = 14, edge_index size = 116976
	Order lifted.
	Paths updated.
k = 15, edge_index size = 156655
	Order lifted.
	Paths updated.
k = 16, edge_index size = 212475
	Order lifted.
	Paths updated.
k = 17, edge_index size = 289993
	Order lifted.
	Paths updated.
k = 18, edge_index size = 395705
	Order lifted.
	Paths updated

In [17]:
sp_lengths[sp_lengths < 100].max()

tensor(31., device='cuda:0')

In [7]:
pp.algorithms.centrality.temporal_closeness_centrality(t, delta=5, normalized=False)

Directed graph with 1910 nodes and 562 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/1417
Processing root 11/1417
Processing root 21/1417
Processing root 31/1417
Processing root 41/1417
Processing root 51/1417
Processing root 61/1417
Processing root 71/1417
Processing root 81/1417
Processing root 91/1417
Processing root 101/1417
Processing root 111/1417
Processing root 121/1417
Processing root 131/1417
Processing root 141/1417
Processing root 151/1417
Processing root 161/1417
Processing root 171/1417
Processing root 181/1417
Processing root 191/1417
Processing root 201/1417
Processing root 211/1417
Processing root 221/1417
Processing root 231/1417
Processing root 241/1417
Processing root 251/1417
Processing root 261/1417
Processing root 271/1417
Processing root 281/1417
Processing root 291/1417
Processing root 301/1417
Processing root 311/1417
Processing root 321/1417
Processing root 331/1417
Processing root 341/1417
Processing root 351/1417
Processing root 361/1

defaultdict(<function pathpyG.algorithms.centrality.temporal_closeness_centrality.<locals>.<lambda>()>,
            {'Y_WY': 14.2,
             'WRBB': 15.916666666666666,
             'WRR_': 5.0,
             'GGRR': 19.642857142857142,
             'WG_R': 15.0,
             'YWGW': 10.333333333333334,
             'YYGG': 12.5,
             'GBGR': 5.0,
             'GRWG': 12.666666666666666,
             'YY_W': 6.0,
             'G_W_': 6.0,
             'G___small': 2.0,
             'YW__': 2.0,
             'YYGGmid': 19.833333333333332,
             '____brood': 6.25,
             'GGWY': 17.0,
             'G_R_': 2.0,
             '____topleft': 11.0,
             'G_GW': 10.5,
             'GYYY': 4.0,
             'WBGW': 8.833333333333332,
             'Y__W': 4.5,
             'YYYY': 3.5,
             'GWRG': 10.0,
             'YGWW': 16.916666666666668,
             'WRWR': 14.333333333333334,
             'G___big': 2.0,
             'GY__': 3.3333333333333335,
   

In [8]:
pp.algorithms.centrality.temporal_betweenness_centrality(t, delta=5, normalized=False)

Directed graph with 1910 nodes and 562 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/1417
Processing root 11/1417
Processing root 21/1417
Processing root 31/1417
Processing root 41/1417
Processing root 51/1417
Processing root 61/1417
Processing root 71/1417
Processing root 81/1417
Processing root 91/1417
Processing root 101/1417
Processing root 111/1417
Processing root 121/1417
Processing root 131/1417
Processing root 141/1417
Processing root 151/1417
Processing root 161/1417
Processing root 171/1417
Processing root 181/1417
Processing root 191/1417
Processing root 201/1417
Processing root 211/1417
Processing root 221/1417
Processing root 231/1417
Processing root 241/1417
Processing root 251/1417
Processing root 261/1417
Processing root 271/1417
Processing root 281/1417
Processing root 291/1417
Processing root 301/1417
Processing root 311/1417
Processing root 321/1417
Processing root 331/1417
Processing root 341/1417
Processing root 351/1417
Processing root 361/1

defaultdict(<function pathpyG.algorithms.centrality.temporal_betweenness_centrality.<locals>.<lambda>()>,
            {'WG_R': 4.0,
             'GR_Y2': 9.0,
             'GRWG': 5.0,
             'GGRR': 5.0,
             'Y_WY': 4.0,
             'YYGGmid': 4.0,
             'WRWR': 1.0,
             'GRYY': 1.0,
             'WBGG': 4.0,
             '____brood': 2.5,
             'GGWY': 4.0,
             'GR__': 3.0,
             'G_R_': 1.0,
             'GWRG': 3.0,
             'WGWB': 8.0,
             '____almost': 2.0,
             'Q': 7.0,
             '_WYW': 2.0,
             'RWWG': 5.0,
             '_W_Y': 5.0,
             'YWW_': 4.0,
             'WRR_': 1.0,
             'YWGW': 1.0,
             'G_GW': 2.0,
             '____topleft': 2.0,
             'WYGG': 4.0,
             'RWGY': 2.0,
             '_R__': 7.0,
             'YYRB': 2.0,
             'GBGR': 1.0,
             '_WWY': 4.0,
             'GGWW': 1.5,
             'YYGG': 3.0,
             'YY_